## Combine All Three For Twitter API Funtions

In [1]:
# Watson NLU gives fear scores, and requires 1 API call each time
# Sarcasm Detector is a standalone model, and can be pickle loaded.
# NLP model for classification. Done after the other two.

In [2]:
# Import the three packages and pickle
import classes
import nlp_model
import pickle as pk
import watson_nlu
import sarcasm_detector
import sys

In [5]:
# Initiate this in the first place.
nlu_api = watson_nlu.get_ibm_nlu()
clf, tv, ps, scores = sarcasm_detector.get_model()


tweet = "Today is gonna be a good good day. #Nope #Sike"
# Given a tweet as tweet.text (Later on I'll check a list of tweets. )
print(watson_nlu.is_fearful(tweet, nlu_api))
processed_tweets = sarcasm_detector.preprocess([tweet],ps)
print(processed_tweets)
clf.predict(tv.transform(processed_tweets))

False
['today is gonna be a good good day nope sike']


/Users/ekremguzelyel/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([1])

In [9]:
# I know this is sarcastic -> Discard and Not Fearful -> Discard
fear_matrix = [watson_nlu.is_fearful(tweet, nlu_api) for tweet in tweets]
processed_tweets = sarcasm_detector.preprocess([tweet], ps)
sarcasm_matrix = clf.predict(tv.transform(processed_tweets))
result_matrix = [1 if (fear_matrix[i] and sarcasm_matrix[i]) else 0 for i in range(len(fear_matrix))]

/Users/ekremguzelyel/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [10]:
result_matrix

[0]

In [31]:
def is_eligible(tweets):
    """Given list of tweets as text return if emergent/eligible for a test 1, else 0.
    If you want to pass only one tweet, add brackets tweet -> [tweet]
    Returns: 
        1-0 matrix of len(tweets)
    """
    fear_matrix = [watson_nlu.is_fearful(tweet, nlu_api) for tweet in tweets]
    processed_tweets = sarcasm_detector.preprocess(tweets, ps)
    sarcasm_matrix = clf.predict(tv.transform(processed_tweets))
    return [1 if (fear_matrix[i] and not sarcasm_matrix[i]) else 0 for i in range(len(fear_matrix))]

In [32]:
tweets = ["tweet", "I am so afraid. The cops just shot a guy. He is bleeding"]
[watson_nlu.is_fearful(tweet, nlu_api) for tweet in tweets]

[False, True]

In [19]:
eligible_tweets = [tweet for i, tweet in enumerate([tweet]) if is_eligible([tweet])[i]] 

/Users/ekremguzelyel/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [20]:
eligible_tweets

[]

In [43]:
def emergent_tweets(tweets):
    "Returns: Emergent tweets with their corresponding Class."
    eligible_tweets = [tweet for i, tweet in enumerate(tweets) if is_eligible(tweets)[i]]
    return eligible_tweets, nlp_model.classify_list(eligible_tweets)

In [44]:
emergent_tweets(["tweet", "I am so afraid. The cops just shot a guy. He is bleeding"])

/Users/ekremguzelyel/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(['I am so afraid. The cops just shot a guy. He is bleeding'], ['police'])

In [51]:
# Clean up

# Watson NLU gives fear scores, and requires 1 API call each time
# Sarcasm Detector is a standalone model, and can be pickle loaded.
# NLP model for classification. Done after the other two completed.

# Import the three packages and pickle
import classes
import nlp_model
import pickle as pk
import watson_nlu
import sarcasm_detector
import sys

# Initiate this in the first place. Add to main
nlu_api = watson_nlu.get_ibm_nlu()
clf, tv, ps, scores = sarcasm_detector.get_model()


def is_eligible(tweets):
    """Given list of tweets that fetched from search api to check if 
    they are eligible classification.
    If you want to pass only one tweet, add brackets tweet -> [tweet]
    
    Params: 
        tweets: list of dictionaries taken from search api 
    Returns: 
        1-0 matrix of len(tweets)
    """
    fear_matrix = [watson_nlu.is_fearful(tweet.text, nlu_api) for tweet in tweets]
    processed_tweets = sarcasm_detector.preprocess(tweets, ps)
    sarcasm_matrix = clf.predict(tv.transform(processed_tweets))
    return [1 if (fear_matrix[i] and not sarcasm_matrix[i]) else 0 for i in range(len(fear_matrix))]

def emergent_tweets(tweets):
    """ Applies classification criteria to the tweets and returns only eligibles.
    Returns: 
        Emergent tweets with text, id, and place vs. corresponding Class.
    """
    eligible_tweets = zip([(tweet.text, tweet.id, tweet.place) for i, tweet in enumerate(tweets) if is_eligible(tweets)[i]])
    
    return eligible_tweets, nlp_model.classify_list(next(eligible_tweets))

In [52]:
emergent_tweets(["sda", "sdada police hack shot gun"])

AttributeError: 'str' object has no attribute 'text'